# Modern C++ : Improved pointers of the standard library

It can be be qualified as improved pointers (smart), objects which behave like pointers (support for the `*` and `->` operators) but which provide additional services, in particular the automatic destruction of the pointed objects.

The problems by using ordinary pointers (raws) are known : :
- the declaration of a pointer does not specify whether it points to a single object or an array,
- it is not known if the pointer is "owner" of the pointed object, ie if it is in charge of its destruction,
- if it is necessary to destroy the pointed object, it is not known whether a simple delete or a more complex procedure is required, and whether it is a single object or an array,
- it is sometimes difficult to guarantee that the destruction is done only once, regardless of the path taken by the program at the runtime,
- it is not possible to test whether the pointer references an existing or already destroyed object.

The Standard Library strives to provide a collection of enhanced pointers that never allow ordinary pointers to be used.

## `std::auto_ptr`

This is the first implementation, in C++03, of a pointer which "owns" the pointed object, ie when the pointer is destroyed, the pointed object is also destroyed.

In [ ]:
%%file tmp.smart-ptr.cpp

#include <string>
#include <iostream>
#include <memory>

void hello()
 {
  std::string * str = new std::string("hello") ;
  std::auto_ptr<std::string> ptr(str) ;
  std::cout << *ptr << std::endl ;
  *ptr += " world !" ;
  std::cout << *ptr << std::endl ;
 }

int main()
 {
  // ...
  hello() ;
  // ...
  hello() ;
  // ...
  return 0 ;
 }

In [ ]:
!rm -f tmp.smart-ptr.exe && g++ -std=c++03 tmp.smart-ptr.cpp -o tmp.smart-ptr.exe

In [ ]:
!./tmp.smart-ptr.exe

This pointer nevertheless suffers from a major flaw: to avoid double destruction, in the event of a copy, the original pointer is modified to be empty, which is not in accordance with the idea that can be made of a copy. One consequence is that you cannot store `auto_ptr` in standard library containers, as these require objects that have normal copy behavior.

In [ ]:
%%file tmp.smart-ptr.cpp

#include <string>
#include <iostream>
#include <vector>
#include <memory>
#include <algorithm>

int main()
 {
  typedef std::auto_ptr<std::string> string_ptr ;
  typedef std::vector<string_ptr> string_vec ;

  string_vec words ;
  words.push_back(string_ptr(new std::string("world"))) ; // DOES NOT COMPILE !
  words.push_back(string_ptr(new std::string("hello"))) ; // DOES NOT COMPILE !
  std::reverse(words.begin(),words.end()) ;

  string_vec::const_iterator word ;
  for ( word = words.begin() ; word != words.end() ; ++word )
   { std::cout<<(**word)<<std::endl ; }
   
  return 0 ;
 }

In [ ]:
!rm -f tmp.smart-ptr.exe && g++ -std=c++03 tmp.smart-ptr.cpp -o tmp.smart-ptr.exe

In fact, it should now not be used anymore, but it is preferable to use `unique_ptr`, which does the same services, only better, taking advantage of the displacement semantics introduced in C++11.

## `std::unique_ptr`

This type of pointers, which may be called "proprietary pointers", looks like the `auto_ptr` type in C++03, but without its flaws.

#### The "move-only pointer"

The owner pointer is designed in such a way that, at any given time, a given object is referenced by only one such pointer. To do this, it uses the "displacement semantics" : it cannot be copied and it cannot be affected by the usual assignment. However, it can be submitted to the displacement constructor or to the assignment by displacement.

Thanks to this property, it can be safely stored in C++11 containers which have been rewritten taking into account the semantics of movement. In the example below, note that if a pointer is created in a separate instruction (word1), it must be transformed into a temporary value before giving it to the vector : the owner pointers can be moved but not copied.

In [ ]:
#include <iostream>
#include <string>
#include <vector>
#include <memory>
{
  using string_ptr = std::unique_ptr<std::string> ;
  std::vector<string_ptr> words ;
  
  string_ptr word1(new std::string("world")) ;

  words.push_back(std::move(word1)) ;
  words.push_back(string_ptr(new std::string("hello"))) ;
  
  std::reverse(words.begin(),words.end()) ;
  for ( auto const & word : words )
   { std::cout<<(*word)<<std::endl ; }
}

In what follows, instances of `std::unique_ptr` will be called "owner pointers".

#### Performance

A proprietary pointer is no larger than a regular pointer, and for most operations it is just as fast. It is therefore an excellent choice by default.

#### Custom destruction

Building a `unique_ptr` also allows to explicitly provide a custom destruction function.

In [ ]:
#include <string>
#include <iostream>
#include <vector>
#include <memory>

In [ ]:
void delete_string( std::string * chaine )
 { std::cout << "Destruction of " << *chaine << std::endl ; delete chaine ; }

In [ ]:
{
  using string_ptr = std::unique_ptr<std::string,decltype(&delete_string)> ;
  std::vector<string_ptr> words ;
  
  words.push_back(string_ptr(new std::string("world"),delete_string)) ;
  words.push_back(string_ptr(new std::string("hello"),delete_string)) ;
  
  std::reverse(words.begin(),words.end()) ;
  for ( auto const & word : words )
   { std::cout<<(*word)<<std::endl ; }
}

#### Performance

In the case of a custom destruction function, the pointer size can increase significantly.

#### Arrays of objects

Out of curiosity, note that there is a form of `unique_ptr` capable of handling an array of objects, instead of a single object, and therefore able to call the `delete []` form upon destruction. But with all the containers available in the standard library, using regular arrays is neither useful nor recommended.

## `shared_ptr`

As its name suggests, unlike the `unique_ptr` type, this type allows the same object to be referenced by multiple pointers. None of them is then the full owner of the object. Its management is based on a reference counter and the pointed object is destroyed when the counter drops to zero.

Subsequently, the instances of `shared_ptr` will be called "co-owner pointers".

#### Garbage collector

Co-owner pointers can be seen as a way to implement the notion of "garbage collection", present in higher level languages, which frees the programmer from the management of the life cycle of objects. If any object is dynamically allocated and then manipulated through co-owner pointers, it will be automatically destroyed when no one is using it, and there is a form of automatic cleaning.

In [ ]:
#include <iostream>
#include <string>
#include <memory>

In [ ]:
struct C { int * data ; } ;

In [ ]:
void delete_int( int * p )
 { std::cout<<"Deleting an int"<<std::endl ; delete p ; }

In [ ]:
void count( std::string const & name, std::shared_ptr<int> const & ptr )
 { std::cout<<name<<" count : "<<ptr.use_count()<<std::endl ; }

In [ ]:
{
  std::cout<<"\n";
    
  std::shared_ptr<int> p1;
  std::shared_ptr<int> p2 (nullptr);
  count("p1",p1) ;
  count("p2",p2) ;

  std::cout<<"\n";

  std::shared_ptr<int> p3 (new int);
  std::shared_ptr<int> p4 (new int);
  count("p3",p3) ;
  count("p4",p4) ;
 
  std::cout<<"\n";

  std::shared_ptr<int> p5 (new int, delete_int);
  std::shared_ptr<int> p6 (p5);
  std::shared_ptr<int> p7 (std::move(p6));
  count("p5",p5) ;
  count("p6",p6) ;
  count("p7",p7) ;
  
  std::cout<<"\n";

  std::shared_ptr<int> p8 (std::unique_ptr<int>(new int));
  count("p8",p8) ;
  
  std::cout<<"\n";

  std::shared_ptr<C> obj (new C);
  std::shared_ptr<int> p9 (obj, obj->data);
  count("p9",p9) ;
  
  std::cout<<"\n";
}

#### Performance

If you are sensitive to the performance of your application, be careful not to overuse co-owner pointers. Indeed, they have a certain cost :
- they are doubled in size, compared to an ordinary pointer, because they also point to a "control block " which notably contains the current number of references on the pointed object
- the creation of the first co-owner pointer to a given object implies the dynamic creation of the "control block" associated with the pointed object.
- increasing or decreasing the number of referrals should be done in a thread-safe way, so a little bite slower.

#### Custom destruction

As with owner pointers, a custom destructor can be specified, which this time does not increase the size of co-owner pointers, because it is stored with the reference count.

#### Warning

Care should be taken not to transform a given ordinary pointer (or a single pointer) into a co-owner pointer only once, then to make copies only from co-owner pointer to co-owner pointer. Indeed, if several co-owner pointers are initialized from the same ordinary pointer, several control blocks will be created, which will lead to a double destruction...

In [ ]:
#include <iostream>
#include <string>
#include <memory>
{
  int * i1 = new int{1} ;
  //...
  std::shared_ptr<int> pa{i1} ;
  //...
  std::shared_ptr<int> pb{i1} ;
  //...
}

To avoid being tempted to transform an ordinary pointer into a co-owner pointer several times in this way, put the result of a call to new directly in the co-owner pointer constructor :

In [ ]:
#include <iostream>
#include <string>
#include <memory>
{
  //...
  std::shared_ptr<int> pa{new int{1}} ;
  //...
  std::shared_ptr<int> pb{pa} ;
  //...
}

Or even better, use the `make_shared` function described later.

## `weak_ptr`

The `shared_ptr` and `unique_ptr` types are not suited to the case of a graph of objects that point to each other, potentially creating cycles. For this case (and for some others), it is necessary to have different pointers, which do not own the pointed objects. This is the goal of the `weak_ptr` class. It is called "weak pointers".

#### Non-proprietary pointers

Weak pointers look like co-owner pointers, but do not affect the reference count of the pointed object. One might be tempted to use ordinary pointers for this task, but the weak pointer has an additional advantage : it is able to tell if the pointed object still exists or if it has been destroyed, because it has access to its "control block". And to have access to this control block, a weak pointer is built from a co-owner pointer :

In [ ]:
#include <iostream>
#include <memory>

In [ ]:
struct Neuron 
 {
  Neuron() { std::cout<<"Construction"<<std::endl ; }
  ~Neuron() { std::cout<<"Destruction"<<std::endl ; }
 } ;

In [ ]:
{
  std::shared_ptr<Neuron> shptr(new Neuron) ;
  std::weak_ptr<Neuron> wptr(shptr) ;
  std::cout<<"Count : "<<shptr.use_count()<<std::endl ;
  std::cout<<"Expired ? "<<wptr.expired()<<std::endl ;
  shptr.reset() ;
  std::cout<<"Count : "<<shptr.use_count()<<std::endl ;
  std::cout<<"Expired ? "<<wptr.expired()<<std::endl ;
}

In addition, the designers of the standard library also wanted the enhanced pointers to be thread safe. As such, it is necessary to prevent that a pointed object disappears between the moment or one would test its existence, and the moment or one would use it. For this purpose, pointers are endowed with an "atomic" method which tests for existence and returns a co-owner pointer, which contains the pointed object if it existed, and nothing otherwise. In a way, the creation of the new co-owner pointer locks the existence of the pointed object :

In [ ]:
#include <iostream>
#include <string>
#include <memory>
#include <vector>

In [ ]:
struct Neuron 
 {
  int m_id ;
  std::vector<std::weak_ptr<Neuron>> m_connexions ;
  
  explicit Neuron( int id ) : m_id(id)
   { std::cout<<"Neuron's construction "<<m_id<<std::endl ; }
  ~Neuron()
   { std::cout<<"Neuron's destruction "<<m_id<<std::endl ; }
   friend std::ostream & operator<<( std::ostream & os, Neuron const & n )
    {
     os<<"neuron "<<n.m_id<<", lié à" ;
     for ( auto c : n.m_connexions )
      {
       auto p = c.lock() ;
       if (p) os<<" "<<p->m_id ;
      }
     return os ;
    }
 } ;

In [ ]:
{
  // crée
  std::vector<std::shared_ptr<Neuron>> v ;
  v.push_back(std::shared_ptr<Neuron>(new Neuron(0)));
  v.push_back(std::shared_ptr<Neuron>(new Neuron(1)));
  v.push_back(std::shared_ptr<Neuron>(new Neuron(2)));
  v.push_back(std::shared_ptr<Neuron>(new Neuron(3)));

  // connecte
  for ( auto n1 : v )
    for ( auto const & n2 : v )
      if ( n1->m_id != n2->m_id )
       { n1->m_connexions.push_back(std::weak_ptr<Neuron>(n2)) ; }    

  // inspecte
  for ( auto neuron : v )
   {
    std::cout
      <<neuron.use_count()
      <<" pointers to "
      <<*neuron
      <<std::endl ;
   }   
  v.pop_back() ;
  v.erase(++(v.begin())) ;
  for ( auto neuron : v )
   {
    std::cout
      <<neuron.use_count()
      <<" pointers to "
      <<*neuron
      <<std::endl ;
   }   
}

#### Performance

The test for the existence of the pointed object is based on the control block associated with this object. This block contains not only the number of co-owner pointers that point to the object, but also the number of weak pointers. When the number of co-owners drops to 0, the object is destroyed, but not if the block, which remains as long as there is still at least one weak pointer, which allows them to detect that the object they point to no longer exists.

## `make_unique` et `make_shared`

Whenever possible, it is recommended that you use this indirect way of creating pointers, rather than calling the `new` operator yourself. This avoids having to write the type of the pointed object twice, and it avoids a possible memory leak in the event of an exception.

```c++
std::unique_ptr<Widget> upw2(new Widget); // without make func
auto upw1(std::make_unique<Widget>());    // with make func

std::shared_ptr<Widget> spw2(new Widget); // without make func
auto spw1(std::make_shared<Widget>());    // with make func

processWidget(std::shared_ptr<Widget>(new Widget), priority()) ; // without make func
processWidget(std::make_shared<Widget>(), priority()) ; // with make func
```

A noter : `make_unique` n'apparait qu'avec C++14.

#### Performance

Moreover, the implementation of `make_shared` is optimized to make a single memory allocation both for the pointed object and for its control block. On the other hand, the memory containing the object and the block can only be returned when all the `shared_ptr` AND all the `weak_ptr` have been destroyed.

#### Limits

The use of `make_unique` or `make_shared` is not appropriate if a custom destructor is declared, if the use of braces construction is required (because these functions use the parenthesis construction), or if  very large objects are present and weak_ptr which outlive for a long time to their objects.

© *CNRS 2021*  
*This document was created by David Chamont and translated by Karim Hasnaoui. It is available under the [License Creative Commons - Attribution - No commercial use - Shared under the conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*